In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !unzip alien-vs-predator-images.zip

In [4]:
import zipfile
with zipfile.ZipFile("/content/alien-vs-predator-images.zip","r") as zip_ref:
    zip_ref.extractall("/content/alien-vs-predator-images")

In [5]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [6]:
import cv2
import os
import pandas as pd
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.xception import Xception
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from tensorflow.python.keras.layers import Dense,Flatten
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import MaxPooling2D
import h5py

In [7]:
path="/content/alien-vs-predator-images/alien-vs-predator-images/data/train/alien/"
os.chdir(path)
im=cv2.imread("0.jpg")

In [8]:
im.shape
os.chdir(os.getcwd())

In [9]:
#training images
d="/content/alien-vs-predator-images/alien-vs-predator-images/data/train/"
#PATH = '../'
checkpoint_dir = '/content/alien-vs-predator-images/alien-vs-predator-images/data/ckpt'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
img_size = 299
channels = 3


classes = os.listdir(d)
print('Classes are:',classes)

paths = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

print('Path is',paths)
nbEntries = []

for i in range(len(classes)):
    #print('Values are:',len(os.listdir(paths[i])))
    nbEntries.append(len(os.listdir(paths[i])))

df = pd.DataFrame({'classes':classes,'entries':nbEntries})

print('Dataframe is:',df)



Classes are: ['predator', 'alien']
Path is ['/content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator', '/content/alien-vs-predator-images/alien-vs-predator-images/data/train/alien']
Dataframe is:     classes  entries
0  predator      347
1     alien      347


In [10]:
train_set = []
train_labels = []

for root,dirs,files in os.walk(d):
    for f in files:
        path = os.path.join(root,f)
        train_set.append(path)
        train_labels.append(root.split(os.path.sep)[-1])

print("length of Total files is:",len(train_set))
print('length of Total labels is:',len(train_labels))

length of Total files is: 694
length of Total labels is: 694


In [11]:
##validation data
d="/content/alien-vs-predator-images/alien-vs-predator-images/data/validation/"

classes = os.listdir(d)
print('Classes are:',classes)

paths = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

print('Path is',paths)
nbEntries = []

for i in range(len(classes)):
    #print('Values are:',len(os.listdir(paths[i])))
    nbEntries.append(len(os.listdir(paths[i])))

df = pd.DataFrame({'classes':classes,'entries':nbEntries})

print('Dataframe is:',df)


Classes are: ['predator', 'alien']
Path is ['/content/alien-vs-predator-images/alien-vs-predator-images/data/validation/predator', '/content/alien-vs-predator-images/alien-vs-predator-images/data/validation/alien']
Dataframe is:     classes  entries
0  predator      100
1     alien      100


In [12]:
val_set = []
val_labels = []

for root,dirs,files in os.walk(d):
    for f in files:
        path = os.path.join(root,f)
        val_set.append(path)
        val_labels.append(root.split(os.path.sep)[-1])

print("length of Total files is:",len(val_set))
print('length of Total labels is:',len(val_labels))

length of Total files is: 200
length of Total labels is: 200


In [13]:
def process_image(img_set):
    processed_image = []
    for i in range(len(img_set)):
        #print(img_set[i])
        temp = cv2.imread(img_set[i],cv2.IMREAD_COLOR)
        print("name:",img_set[i])
        print("\n\t shape:",temp.shape)
        temp = cv2.resize(temp,(img_size,img_size))
        processed_image.append(temp)
    return processed_image


In [14]:
data_train = process_image(train_set)
data_val = process_image(val_set)

name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/117.jpg

	 shape: (312, 162, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/236.jpg

	 shape: (275, 183, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/83.jpg

	 shape: (168, 300, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/337.jpg

	 shape: (262, 192, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/114.jpg

	 shape: (206, 245, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/245.jpg

	 shape: (272, 186, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/242.jpg

	 shape: (285, 177, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/data/train/predator/328.jpg

	 shape: (163, 309, 3)
name: /content/alien-vs-predator-images/alien-vs-predator-images/

In [15]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(list(classes))

x_train = np.array(data_train)
y_train = lb.transform(np.array(train_labels))
#print('Y train is',y_train)

x_val = np.array(data_val)
y_val = lb.transform(np.array(val_labels))
#print('Y Validation is',y_val)

In [16]:
def modelArchitecture():
    base_model = Xception(weights='imagenet', include_top=False,input_shape = (img_size,img_size,channels))
    model = Sequential()
    #model.add(Conv2D(64, (3, 3)),input_shape=(img_size,img_size,channels))
    
    model.add(base_model)
    '''
    model.add(Conv2D(64, (3, 3),activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3),activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Conv2D(256, (3, 3),activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    '''
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation='relu',name = 'dense'))
    model.add(Dropout(0.5))
    model.add(layers.Dense(len(classes),activation='softmax',name = 'predictions'))
    model.summary()

    # from keras.optimizers import Adam
    learning_rate = 0.0001

    model.compile(optimizer = Adam(learning_rate =learning_rate),loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

    return model


In [17]:
def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no ckpt available.
    checkpoints = [checkpoint_dir + '\\' + name
                   for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print('Restoring from', latest_checkpoint)
        return load_model(latest_checkpoint)
    print('Creating a new model')
    return modelArchitecture()


In [18]:
batch_size=32

model = make_or_restore_model()
EarlyStopping_callback= EarlyStopping(monitor='val_accuracy', patience=5)

checkpoint = ModelCheckpoint(filepath="/content/alien-vs-predator-images/alien-vs-predator-images/data/ckpt/model1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,EarlyStopping_callback]
#model.fit(datagen.flow(x_train,y_train,batch_size=8),steps_per_epoch=len(x_train), epochs=100)
model.fit(x_train, y_train, epochs=100, batch_size=8,callbacks=callbacks_list,verbose = 1,validation_data = (x_val,y_val))



Creating a new model
83689472/83683744 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten (Flatten)            (None, 204800)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               26214528  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
predictions (Dense)          (None, 2)                 258       
Total params: 47,076,266
Trainable params: 47,021,738
Non-trainable params: 54,528
_________________________________________________________________
Epoch 1/100
87/87 [

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
87/87 [==============================] - 27s 312ms/step - loss: 0.0868 - accuracy: 0.9669 - val_loss: 0.0913 - val_accuracy: 0.9650

Epoch 00002: val_accuracy improved from 0.94000 to 0.96500, saving model to /content/alien-vs-predator-images/alien-vs-predator-images/data/ckpt/model1.h5
Epoch 3/100
87/87 [==============================] - 29s 332ms/step - loss: 0.0999 - accuracy: 0.9683 - val_loss: 0.0888 - val_accuracy: 0.9650

Epoch 00003: val_accuracy did not improve from 0.96500
Epoch 4/100
87/87 [==============================] - 29s 335ms/step - loss: 0.0828 - accuracy: 0.9784 - val_loss: 0.1543 - val_accuracy: 0.9400

Epoch 00004: val_accuracy did not improve from 0.96500
Epoch 5/100
87/87 [==============================] - 29s 333ms/step - loss: 0.0312 - accuracy: 0.9899 - val_loss: 0.2317 - val_accuracy: 0.9500

Epoch 00005: val_accuracy did not improve from 0.96500
Epoch 6/100
87/87 [==============================] - 29s 334ms/step - loss: 0.0381 - accuracy: 0.991

In [19]:
val_path="/content/alien-vs-predator-images/alien-vs-predator-images/data/validation/alien/0.jpg"

data_test = process_image([val_path])
x_test = np.array(data_test)
predict = model.predict_classes(x_test, verbose=1)
print("predicted value",predict)

name: /content/alien-vs-predator-images/alien-vs-predator-images/data/validation/alien/0.jpg

	 shape: (183, 275, 3)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


1/1 [==============================] - 1s 872ms/step
predicted value [0]


In [20]:
test_results = {}

test_results['model'] = model.evaluate(
   x_val,y_val, verbose=0)

print(f" Accuracy: {test_results}")


 Accuracy: {'model': [0.2822059690952301, 0.9449999928474426]}
